# 3D Tomographic Reconstruction with GPU Acceleration

This notebook demonstrates the complete workflow for reconstructing 3D volumetric data from 2D projections with GPU acceleration. The implementation supports:

1. **ASTRA Toolbox** - specialized high-performance GPU tomography (fastest)
2. **CuPy GPU** - general-purpose GPU acceleration
3. **CPU** - fallback implementation

## Overview
1. Data loading and metadata extraction
2. Preprocessing and filtering
3. Accelerated reconstruction implementation
4. Performance comparison
5. 3D visualization and export


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.ndimage as ndimage
from skimage import io, transform, filters
import os
import sys
from tqdm import tqdm
import plotly.graph_objects as go
import time

# Add the src directory to the path
sys.path.append('../src')

# Import our custom modules
from preprocessing import preprocess_projections
from visualization import show_projections, show_sinogram, show_volume_slices, show_volume_3slice, render_surface_plotly
from utils import load_projections_with_metadata, create_projection_geometry, save_numpy_as_vtk, save_volume_as_tiff_stack

# Import the accelerated reconstruction module (auto-selects best implementation)
from accelerated import (
    filtered_backprojection, 
    art_reconstruction, 
    sirt_reconstruction, 
    fdk_reconstruction,
    print_gpu_info,
    estimate_required_gpu_memory
)

## GPU Acceleration Information

Check what acceleration options are available on this system:

In [ ]:
# Print GPU acceleration information
print_gpu_info()

## 1. Data Loading and Metadata Extraction

In this section, we load the 2D projections and associated metadata from text files. The system can automatically find and parse metadata files that accompany TIFF images.

In [ ]:
# Define paths
data_path = '../data/raw/'
processed_path = '../data/processed/'
reconstructions_path = '../data/reconstructions/'

# Create directories if they don't exist
for path in [processed_path, reconstructions_path]:
    if not os.path.exists(path):
        os.makedirs(path)

# Parameters for loading data
file_pattern = '*.tif*'  # Pattern for TIFF files
angle_pattern = '_([\d.]+)deg'  # Pattern to extract angles from filenames (as fallback)

# Check if data exists
if os.path.exists(data_path) and any(f.endswith(('.tif', '.tiff')) for f in os.listdir(data_path)):
    # Load projections with metadata
    print("Loading projections and metadata from files...")
    projections, angles, metadata = load_projections_with_metadata(data_path, file_pattern, angle_pattern)
    
    print(f"Loaded {len(projections)} projections with shape {projections[0].shape}")
    print(f"Angle range: {angles.min():.2f}° to {angles.max():.2f}°")
    
    # Print available metadata
    print("\nMetadata found:")
    for key, value in metadata.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for subkey, subvalue in value.items():
                print(f"    {subkey}: {subvalue}")
        else:
            print(f"  {key}: {value}")
    
    # Display sample projections
    indices = [0, len(projections)//3, 2*len(projections)//3, len(projections)-1]
    show_projections(projections, indices)
    
    # Display a sinogram
    show_sinogram(projections)
else:
    print("No data found in the raw data directory. Creating a simulation phantom for demonstration...")
    
    # Create a simple phantom (a cube with a sphere inside)
    phantom_size = 128
    phantom = np.zeros((phantom_size, phantom_size, phantom_size))
    
    # Add a cube
    margin = 20
    phantom[margin:-margin, margin:-margin, margin:-margin] = 0.5
    
    # Add a sphere
    x, y, z = np.ogrid[:phantom_size, :phantom_size, :phantom_size]
    center = phantom_size // 2
    radius = phantom_size // 4
    sphere = (x - center)**2 + (y - center)**2 + (z - center)**2 <= radius**2
    phantom[sphere] = 1.0
    
    # Create projection angles
    n_angles = 180
    angles = np.linspace(0, 180, n_angles, endpoint=False)
    
    # Create projections
    projections = np.zeros((n_angles, phantom_size, phantom_size))
    
    for i, angle in enumerate(angles):
        # Simple parallel beam projection (sum along rotated z-axis)
        rotated = ndimage.rotate(phantom, angle, axes=(0, 1), reshape=False, order=1)
        projections[i] = np.sum(rotated, axis=0)
    
    # Normalize projections to [0, 1]
    projections = (projections - projections.min()) / (projections.max() - projections.min())
    
    # Create sample metadata
    metadata = {
        'source_detector_distance': 1000.0,
        'source_object_distance': 500.0,
        'exposure_time': 100.0,
        'geometry': {
            'source_origin_dist': 500.0,
            'origin_detector_dist': 500.0
        }
    }
    
    # Save the phantom and projections
    np.save(f"{reconstructions_path}/phantom.npy", phantom)
    np.save(f"{processed_path}/simulated_projections.npy", projections)
    
    # Display sample projections
    indices = [0, n_angles//3, 2*n_angles//3, n_angles-1]
    show_projections(projections, indices)
    
    # Display a sinogram
    show_sinogram(projections)

In [ ]:
# Preprocess the projections
processed_projections = preprocess_projections(
    projections,
    angles=angles,
    normalize=True,
    denoise=True,
    remove_rings=True,
    correct_rotation=True,
    log_transform=True
)

# Display preprocessed projections
show_projections(processed_projections, indices)

## 2. Geometric Setup from Metadata

Define the geometry of our imaging setup using the metadata.

In [ ]:
# Use geometry information from metadata if available
if 'geometry' in metadata:
    source_origin_dist = metadata['geometry']['source_origin_dist']
    origin_detector_dist = metadata['geometry']['origin_detector_dist']
    print(f"Using geometry from metadata:")
else:
    # Default values if not in metadata
    source_origin_dist = 500.0  # mm (distance from source to rotation center)
    origin_detector_dist = 500.0  # mm (distance from rotation center to detector)
    print(f"Using default geometry (metadata not available):")

print(f"  Source to rotation center distance: {source_origin_dist} mm")
print(f"  Rotation center to detector distance: {origin_detector_dist} mm")

detector_shape = projections.shape[1:3]  # (height, width) of detector

# Setup projection geometry
geometry = create_projection_geometry(
    angles,
    detector_shape,
    source_origin_dist,
    origin_detector_dist
)

# Define the size of the reconstruction volume
volume_size = (detector_shape[0], detector_shape[0], detector_shape[0])  # cubic volume
print(f"Reconstruction volume size: {volume_size}")

# Estimate required GPU memory
required_gb = estimate_required_gpu_memory(volume_size, projections.shape)
print(f"Estimated GPU memory required: {required_gb:.2f} GB")

## 3. Accelerated Reconstruction 

Compare different reconstruction algorithms and acceleration methods.

### 3.1 Filtered Back Projection (FBP)

In [ ]:
# Run FBP with automatic acceleration (ASTRA > GPU > CPU)
print("Starting Filtered Back Projection reconstruction...")
start_time = time.time()

fbp_volume = filtered_backprojection(
    processed_projections,
    angles,
    volume_size
)

end_time = time.time()
print(f"FBP reconstruction completed in {end_time - start_time:.2f} seconds.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/fbp_volume.npy", fbp_volume)
save_numpy_as_vtk(fbp_volume, f"{reconstructions_path}/fbp_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(fbp_volume)

### 3.2 Algebraic Reconstruction Technique (ART)

In [ ]:
# Run ART with automatic acceleration
print("Starting ART reconstruction...")
start_time = time.time()

art_volume = art_reconstruction(
    processed_projections,
    angles,
    volume_size,
    iterations=5  # ART is iterative, so we specify the number of iterations
)

end_time = time.time()
print(f"ART reconstruction completed in {end_time - start_time:.2f} seconds.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/art_volume.npy", art_volume)
save_numpy_as_vtk(art_volume, f"{reconstructions_path}/art_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(art_volume)

### 3.3 SIRT Reconstruction

In [ ]:
# Run SIRT with automatic acceleration
print("Starting SIRT reconstruction...")
start_time = time.time()

sirt_volume = sirt_reconstruction(
    processed_projections,
    angles,
    volume_size,
    iterations=20
)

end_time = time.time()
print(f"SIRT reconstruction completed in {end_time - start_time:.2f} seconds.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/sirt_volume.npy", sirt_volume)
save_numpy_as_vtk(sirt_volume, f"{reconstructions_path}/sirt_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(sirt_volume)

### 3.4 FDK Reconstruction (Cone Beam)

In [ ]:
# Run FDK with automatic acceleration
print("Starting FDK reconstruction...")
start_time = time.time()

fdk_volume = fdk_reconstruction(
    processed_projections,
    geometry,
    volume_size
)

end_time = time.time()
print(f"FDK reconstruction completed in {end_time - start_time:.2f} seconds.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/fdk_volume.npy", fdk_volume)
save_numpy_as_vtk(fdk_volume, f"{reconstructions_path}/fdk_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(fdk_volume)

## 4. Acceleration Method Comparison

Compare the performance of different acceleration methods for FBP:

In [ ]:
# Compare the performance of different acceleration methods for FBP
methods = []
times = []

# Only run if volume is not too large to avoid memory issues
if np.prod(volume_size) <= 256**3:  # Skip for very large volumes 
    # CPU implementation
    print("\nRunning CPU implementation...")
    start_time = time.time()
    
    cpu_volume = filtered_backprojection(
        processed_projections,
        angles,
        volume_size,
        use_gpu=False,
        use_astra=False
    )
    
    cpu_time = time.time() - start_time
    print(f"CPU time: {cpu_time:.2f} seconds")
    methods.append("CPU")
    times.append(cpu_time)
    
    # CuPy GPU implementation
    try:
        print("\nRunning CuPy GPU implementation...")
        start_time = time.time()
        
        gpu_volume = filtered_backprojection(
            processed_projections,
            angles,
            volume_size,
            use_gpu=True,
            use_astra=False
        )
        
        gpu_time = time.time() - start_time
        print(f"CuPy GPU time: {gpu_time:.2f} seconds")
        print(f"Speedup: {cpu_time / gpu_time:.1f}x")
        methods.append("CuPy GPU")
        times.append(gpu_time)
    except Exception as e:
        print(f"CuPy GPU implementation failed: {e}")
    
    # ASTRA implementation
    try:
        print("\nRunning ASTRA Toolbox implementation...")
        start_time = time.time()
        
        astra_volume = filtered_backprojection(
            processed_projections,
            angles,
            volume_size,
            use_gpu=True,
            use_astra=True
        )
        
        astra_time = time.time() - start_time
        print(f"ASTRA time: {astra_time:.2f} seconds")
        print(f"Speedup: {cpu_time / astra_time:.1f}x")
        methods.append("ASTRA")
        times.append(astra_time)
    except Exception as e:
        print(f"ASTRA implementation failed: {e}")
    
    # Plot performance comparison
    if len(methods) > 1:
        plt.figure(figsize=(10, 6))
        bars = plt.bar(methods, times)
        plt.title("Reconstruction Performance Comparison")
        plt.ylabel("Time (seconds)")
        plt.grid(axis='y', alpha=0.3)
        
        # Add speedup annotations
        for i, time_val in enumerate(times):
            if i > 0:  # Skip the first bar (CPU)
                speedup = times[0] / time_val
                plt.text(i, time_val + 0.05 * max(times), 
                       f"{speedup:.1f}x faster", 
                       ha='center', va='bottom')
            plt.text(i, time_val / 2, f"{time_val:.2f}s", ha='center', va='center', color='white', fontweight='bold')
        
        plt.savefig(f"{reconstructions_path}/acceleration_comparison.png", dpi=300)
        plt.show()
else:
    print("Volume is too large for performance comparison. Skipping benchmark.")

## 5. 3D Visualization and Export

Visualize the 3D reconstruction results and export to common 3D file formats.

In [ ]:
# Create interactive 3D visualization with Plotly
# Adjust threshold as needed (higher threshold for sparser result)
try:
    threshold = 0.5 * fbp_volume.max()
    fig = render_surface_plotly(fbp_volume, threshold=threshold, opacity=0.7)
    fig.write_html(f"{reconstructions_path}/fbp_isosurface.html")
    fig.show()
except Exception as e:
    print(f"Error creating 3D visualization: {e}")

In [ ]:
# Export as TIFF stack for compatibility with other 3D software
save_volume_as_tiff_stack(fbp_volume, f"{reconstructions_path}/fbp_slices", "slice")
print(f"Saved TIFF stack to {reconstructions_path}/fbp_slices/")

# Also save as VTK file with metadata-based pixel spacing (if available)
pixel_size = metadata.get('pixel_size', 1.0) if isinstance(metadata, dict) else 1.0
spacing = (pixel_size, pixel_size, pixel_size)
save_numpy_as_vtk(fbp_volume, f"{reconstructions_path}/fbp_volume_scaled.vti", spacing=spacing)
print(f"Saved scaled VTK file with pixel size = {pixel_size} mm")

## 6. Result Comparison

Compare the quality of different reconstruction algorithms.

In [ ]:
# Compare middle slices of different reconstruction methods
methods = ['FBP', 'ART', 'SIRT', 'FDK']
volumes = [fbp_volume, art_volume, sirt_volume, fdk_volume]

# Get middle slice
mid_z = volume_size[2] // 2

plt.figure(figsize=(16, 4))
for i, (method, vol) in enumerate(zip(methods, volumes)):
    plt.subplot(1, 4, i+1)
    plt.imshow(vol[:, :, mid_z], cmap='gray')
    plt.title(f"{method} Reconstruction")
    plt.colorbar()
plt.tight_layout()
plt.savefig(f"{reconstructions_path}/method_comparison.png", dpi=300)
plt.show()

## 7. Conclusion

In this notebook, we've demonstrated the benefits of GPU acceleration for tomographic reconstruction:

1. **ASTRA Toolbox** provides the fastest performance, as it's specifically optimized for tomography
2. **CuPy GPU** implementation offers good general-purpose acceleration
3. **CPU** implementation serves as a reliable fallback

We've also shown how to read metadata from accompanying text files in various formats, and how to export the 3D reconstruction results in formats compatible with common 3D visualization software.